Issues:

1) test_image_14 : two squares treated as one, due to which undefined centroid key value in color dict, also the only **orange square** being detected is giving an answer- **Red** *(refer issue and issue1 images)*

2) test_image_15 : **one orange triangle** detected as **blue** triangle *(refer issue2 image)*

Maybe the issue lies in orange contouring, but I tried every permutation to get it through

In [86]:
import cv2
import numpy as np
import os
##############################################################

################# ADD UTILITY FUNCTIONS HERE #################

def centroid(contour):
    """
    Purpose:
    ---
    This function takes contour as an argument and returns a list containing
    the x and y values of the centroid

    Input Arguments:
    ---
    `contour` : [ numpy array ]
            numpy array of (x, y) coordinates of boundary points of the object.

    Returns:
    ---
    `centroid` : [ list ]
            list of (x,y) coordinates of the centroid of the contours
    """
    
    centroid = []

    M = cv2.moments(contour)
    if M['m00'] != 0.0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])

        centroid.append((cx,cy))

    return centroid


def colors_detected(img):
    """
    Purpose:
    ---
    This function takes the image as argument and returns a dictionary
    denoting the color of the shapes in the image.

    Input Arguments:
    ---
    `img` : [ numpy array ]
            numpy array of image returned by cv2 library

    Returns:
    ---
    `detected_colors` : {dictionary}
            dictionary containing details of colors present in image
    """
    
    detected_colors = {}
                
    # Convert BGR to HSV
    hsv_frame = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # defining color ranges

    # red color range
    low_red = np.array([0, 150, 50], np.uint8)
    high_red = np.array([10, 255, 255], np.uint8)
    red_mask = cv2.inRange(hsv_frame, low_red, high_red)

    # blue color range
    low_blue = np.array([94, 80, 2], np.uint8)
    high_blue = np.array([126, 255, 255], np.uint8)
    blue_mask = cv2.inRange(hsv_frame, low_blue, high_blue)

    # green color range
    low_green = np.array([36, 25, 25], np.uint8)
    high_green = np.array([70, 255, 255], np.uint8)
    green_mask = cv2.inRange(hsv_frame, low_green, high_green)

    # orange color range
    low_orange = np.array([10, 150, 50], np.uint8)
    high_orange = np.array([25, 255, 255], np.uint8)
    orange_mask = cv2.inRange(hsv_frame, low_orange, high_orange)

    # Morphological Transform: Dilation, to remove noises from the images and performing bitwise_and function to specifically detect a particular color.

    kernal = np.ones((6, 6), "uint8")
      
    # For red color
    red_mask = cv2.dilate(red_mask, kernal)
      
    # For green color
    green_mask = cv2.dilate(green_mask, kernal)
      
    # For blue color
    blue_mask = cv2.dilate(blue_mask, kernal)

    # For orange color
    orange_mask = cv2.dilate(orange_mask, kernal)
    
    # Contours for color detection

    # Creating contour to track red color
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Creating contour to track green color
    green_contours, _ = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  
    # Creating contour to track blue color
    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    # Creating contour to track orange color
    orange_contours, _ = cv2.findContours(orange_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # RED
    for contour in red_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]

            detected_colors[cx,cy] = ['Red']
    
    #GREEN
    for contour in green_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]


            detected_colors[cx,cy] = ['Green']
        
    #BLUE
    for contour in blue_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]


            detected_colors[cx,cy] = ['Blue']
        
    #ORANGE
    for contour in orange_contours:
        area = cv2.contourArea(contour)
        if(area > 300):
            centroids = centroid(contour)

            # centroid    
            cx = centroids[0][0]
            cy = centroids[0][1]


            detected_colors[cx,cy] = ['Orange']

    return detected_colors


def shapes(img):
    
    img_shapes = {}
    # reading image and converting it to grayscale
#     image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Thresholding
    _, threshold = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)

    # Contouring
    contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #to loop over the contours
    i=0
    for cnt in contours:
    
        # ignoring first counter because 
        # findcontour function detects whole image as one shape
        if i==0:
            i=1
            continue

        # approximating
        approx = cv2.approxPolyDP(cnt, 0.025*cv2.arcLength(cnt, True), True)


        centroids = centroid(cnt)

        # centroid    
        cx = centroids[0][0]
        cy = centroids[0][1]
        
        # putting shape name
        if len(approx) == 3:
            img_shapes[cx,cy] = ['Triangle']
  
        elif len(approx) == 4:
        
            # for distinguishing between rectangle and square
            x, y, w, h = cv2.boundingRect(approx)

            aspectRatio = float (w)/h
            if aspectRatio >= 0.95 and aspectRatio <= 1.05:
                img_shapes[cx,cy] = ['Square']
            else:
                img_shapes[cx,cy] = ['Rectangle']

        elif (len(approx) == 5):
             img_shapes[cx,cy] = ['Pentagon']
  
        else:
             img_shapes[cx,cy] = ['Circle']

    return img_shapes


##############################################################

def detect_shapes(img):

	"""
	Purpose:
	---
	This function takes the image as an argument and returns a nested list
	containing details of colored (non-white) shapes in that image

	Input Arguments:
	---
	`img` :	[ numpy array ]
			numpy array of image returned by cv2 library

	Returns:
	---
	`detected_shapes` : [ list ]
			nested list containing details of colored (non-white) 
			shapes present in image
	
	Example call:
	---
	shapes = detect_shapes(img)
	"""    
	detected_shapes = []

	##############	ADD YOUR CODE HERE	##############
	
	color = colors_detected(img)

	shape = shapes(img)

# 	print(color)
    
# 	print(shape)
    
	keys_color = list(color.keys())
    
	keys_shape = list(shape.keys())
    
# 	print(keys_color)
    
# 	print(keys_shape)
    
	details_list = []

	for key_shp in keys_shape:
# 		print(key_shp[0])
# 		print(key_shp[1])
		details_list = []
		cx_key = key_shp[0]
		cy_key = key_shp[1]
        
        # error inclusion
		upper_cx = 1.01 * cx_key  
#         upper_cx = cx_key + 0.01 * cx_key
		lower_cx = 0.99 * cx_key
    
		upper_cy = 1.01 * cy_key  
		lower_cy = 0.99 * cy_key

		for key_clr in keys_color:
			cx_k = key_clr[0]
			cy_k = key_clr[1]
            

			if (lower_cx <= cx_k <= upper_cx or lower_cy <= cy_k <= upper_cy):
				temp1 = (cx_k, cy_k)
				temp2 = (cx_key, cy_key)
# 				print(temp1)
# 				print(temp2)
				details_list.append(color.get(temp1)[0])
				details_list.append(shape.get(temp2)[0])
				details_list.append(temp2)
				detected_shapes.append(details_list)
				if len(details_list) == 3 :
					break
# 				print(detected_shapes)
    
	for detected in detected_shapes:
		colour = detected[0]
		shape = detected[1]
		coordinates = detected[2]
		cv2.putText(img, str((colour, shape)),coordinates, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
		cv2.imshow("labeled_image", img)
		cv2.waitKey(0)
		cv2.destroyAllWindows()
	##################################################

    
    
	return detected_shapes

In [87]:
def get_labeled_image(img, detected_shapes):
	######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
	"""
	Purpose:
	---
	This function takes the image and the detected shapes list as an argument
	and returns a labelled image

	Input Arguments:
	---
	`img` :	[ numpy array ]
			numpy array of image returned by cv2 library

	`detected_shapes` : [ list ]
			nested list containing details of colored (non-white) 
			shapes present in image

	Returns:
	---
	`img` :	[ numpy array ]
			labelled image
	
	Example call:
	---
	img = get_labeled_image(img, detected_shapes)
	"""
	######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########    

	for detected in detected_shapes:
		colour = detected[0]
		shape = detected[1]
		coordinates = detected[2]
		cv2.putText(img, str((colour, shape)),coordinates, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
	return img

In [88]:
img = cv2.imread('test_images/test_image_1.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Red', 'Circle', (588, 370)], ['Green', 'Square', (325, 145)]]


In [89]:
img = cv2.imread('test_images/test_image_2.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Blue', 'Circle', (274, 348)], ['Blue', 'Pentagon', (648, 420)], ['Green', 'Triangle', (766, 165)], ['Red', 'Rectangle', (301, 118)]]


In [90]:
img = cv2.imread('test_images/test_image_3.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Green', 'Rectangle', (467, 467)], ['Blue', 'Square', (425, 272)], ['Green', 'Circle', (290, 110)], ['Red', 'Triangle', (643, 194)]]


In [91]:
img = cv2.imread('test_images/test_image_4.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 ...

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]]: 
[['Blue', 'Square', (325, 451)], ['Red', 'Rectangle', (249, 199)], ['Blue', 'Circle', (588, 226)]]


In [92]:
img = cv2.imread('test_images/test_image_5.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 ...

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]]: 
[['Red', 'Rectangle', (648, 398)], ['Green', 'Circle', (308, 370)], ['Blue', 'Circle', (631, 149)], ['Blue', 'Square', (183, 105)]]


In [93]:
img = cv2.imread('test_images/test_image_6.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 ...

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]]: 
[['Blue', 'Circle', (274, 348)], ['Blue', 'Pentagon', (648, 420)], ['Green', 'Triangle', (766, 165)], ['Red', 'Rectangle', (301, 118)]]


In [94]:
img = cv2.imread('test_images/test_image_7.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Blue', 'Square', (183, 491)], ['Blue', 'Pentagon', (545, 457)], ['Red', 'Rectangle', (648, 199)], ['Red', 'Circle', (308, 226)], ['Green', 'Pentagon', (103, 108)]]


In [95]:
img = cv2.imread('test_images/test_image_8.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Blue', 'Circle', (295, 503)], ['Red', 'Pentagon', (517, 352)], ['Red', 'Pentagon', (160, 234)], ['Green', 'Triangle', (660, 117)], ['Blue', 'Square', (101, 86)]]


In [96]:
img = cv2.imread('test_images/test_image_9.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Blue', 'Circle', (295, 503)], ['Blue', 'Pentagon', (517, 352)], ['Green', 'Rectangle', (197, 246)], ['Green', 'Triangle', (660, 117)], ['Blue', 'Square', (101, 86)]]


In [97]:
img = cv2.imread('test_images/test_image_10.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Red', 'Circle', (788, 476)], ['Red', 'Rectangle', (420, 453)], ['Red', 'Circle', (107, 340)], ['Blue', 'Triangle', (184, 212)], ['Blue', 'Triangle', (91, 89)]]


In [98]:
img = cv2.imread('test_images/test_image_11.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 ...

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]]: 
[['Blue', 'Rectangle', (467, 467)], ['Blue', 'Square', (425, 272)], ['Red', 'Circle', (290, 110)], ['Red', 'Triangle', (643, 194)]]


In [99]:
img = cv2.imread('test_images/test_image_12.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 ...

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]

 [[176 228 239]
  [176 228 239]
  [176 228 239]
  ...
  [176 228 239]
  [176 228 239]
  [176 228 239]]]: 
[['Red', 'Circle', (788, 476)], ['Red', 'Rectangle', (420, 453)], ['Red', 'Circle', (107, 340)], ['Blue', 'Triangle', (184, 212)], ['Blue', 'Triangle', (91, 89)]]


In [100]:
img = cv2.imread('test_images/test_image_13.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Orange', 'Triangle', (452, 504)], ['Green', 'Square', (778, 482)], ['Blue', 'Rectangle', (650, 401)], ['Red', 'Circle', (529, 325)], ['Red', 'Rectangle', (121, 220)], ['Blue', 'Triangle', (646, 199)]]


In [101]:
img = cv2.imread('test_images/test_image_14.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Blue', 'Triangle', (254, 340)], ['Red', 'Square', (308, 248)], ['Red', 'Circle', (215, 247)], ['Red', 'Circle', (161, 183)], ['Red', 'Circle', (101, 101)]]


In [102]:
img = cv2.imread('test_images/test_image_15.png')
print("For image" + str(img) + ": ")
print(detect_shapes(img))

For image[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]: 
[['Blue', 'Circle', (82, 509)], ['Blue', 'Triangle', (601, 506)], ['Blue', 'Circle', (317, 320)], ['Orange', 'Triangle', (602, 234)], ['Red', 'Square', (201, 218)], ['Blue', 'Circle', (292, 161)], ['Red', 'Square', (395, 165)], ['Red', 'Square', (203, 98)], ['Blue', 'Circle', (793, 96)]]
